In [72]:
import json, requests
from operator import itemgetter
import pyarabic.araby as araby

## Clean Surah Object

In [3]:
with open('surahAdj.json', 'rb') as file:
    f = json.load(file)
f = dict(f)

FileNotFoundError: [Errno 2] No such file or directory: 'surahAdj.json'

In [6]:
newClean = []
for ch in f['chapters']:
    newClean.append({
        'makkah': bool(ch['revelation_place'] == 'makkah'),
        'name_simple': ch['name_simple'],
        'name_arabic': ch['name_arabic'],
        'verses_count': ch['verses_count'],
    })
newClean = {'chapters': newClean}
newClean['chapters'][:2]

[{'makkah': False,
  'name_simple': 'Al-Fatihah',
  'name_arabic': 'الفاتحة',
  'verses_count': 7},
 {'makkah': False,
  'name_simple': 'Al-Baqarah',
  'name_arabic': 'البقرة',
  'verses_count': 286}]

In [7]:
with open('surahAdj.json', 'w') as file:
    json.dump(newClean, file)

## Quran Pages

In [4]:
all_pages = []
for page_num in range(1, 605):
    obj = {}
    res = requests.get(f'https://api.quran.com/api/v4/verses/by_page/{page_num}')
    js = res.json()
    for i, v in enumerate(js['verses']):
        # print(i, v)
        js['verses'][i] = {
            'verse_key': v['verse_key'],
            'juz_number': v['juz_number'],
        }
    all_pages.append(js['verses'])


KeyboardInterrupt: 

In [21]:
all_pages[:1]

[[{'verse_key': '1:1', 'juz_number': 1},
  {'verse_key': '1:2', 'juz_number': 1},
  {'verse_key': '1:3', 'juz_number': 1},
  {'verse_key': '1:4', 'juz_number': 1},
  {'verse_key': '1:5', 'juz_number': 1},
  {'verse_key': '1:6', 'juz_number': 1},
  {'verse_key': '1:7', 'juz_number': 1}]]

In [22]:
with open('versesPerPage.json', 'w') as file:
    json.dump({"pages": all_pages}, file)

### Add mid page ayah

In [105]:
# read quran text file
with open('data.json', 'rb') as file:
    quranData = json.load(file)
quranData = dict(quranData)
quranData['i'][0]

{'title': 'سورة الفاتحة',
 'basmallah': 'بِسْمِ اللَّهِ الرَّحْمَنِ الرَّحِيمِ (1)\n\n',
 'surah': ['بِسْمِ اللَّهِ الرَّحْمَنِ الرَّحِيمِ',
  'الْحَمْدُ لِلَّهِ رَبِّ الْعَالَمِينَ',
  'الرَّحْمَنِ الرَّحِيمِ',
  'مَالِكِ يَوْمِ الدِّينِ',
  'إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُ',
  'اهْدِنَا الصِّرَاطَ الْمُسْتَقِيمَ',
  'صِرَاطَ الَّذِينَ أَنْعَمْتَ عَلَيْهِمْ غَيْرِ الْمَغْضُوبِ عَلَيْهِمْ وَلَا الضَّالِّينَ']}

In [133]:
### get verse text
def getVerse(verse):
    surah, ayah = [int(i) for i in verse.split(':')]
    verseText = quranData['i'][surah - 1]['surah'][ayah - 1]
    return [surah, ayah, verseText]
### get last ayah in a surah
maxAyah = lambda surahNum: len(quranData['i'][surahNum]['surah'])
###
def snapAccumulator(verses, value):
    minimum_list = [abs(verse['accumulator'] - value) for verse in verses]
    return min(enumerate(minimum_list), key=itemgetter(1))[0]

In [130]:
### calculate accumulator for each verse
class CalculateAccumulator:
    def __init__(self):
        self.lastAcc = 0
    def calc(self, verse):
        text = normalizeText(verse['text'])
        verse['accumulator'] = self.lastAcc + len(text)
        self.lastAcc = verse['accumulator']
        return verse
### remove all unwanted characters
def normalizeText(text):
    text = araby.strip_tashkeel(text)
    text = araby.strip_tatweel(text)
    text = text.replace(" ", "")
    return text

In [134]:
### calculate middle ayah
def getMiddle(fromVerse='112:1', toVerse='114:6'):
    fromSurah, fromAyah, fromVerse = getVerse(fromVerse)
    toSurah, toAyah, toVerse = getVerse(toVerse)
    # get all ayahs
    allVerses = []
    for surahI in range(fromSurah - 1, toSurah):
        toInThisSurah = toAyah if surahI == toSurah - 1 else maxAyah(surahI)
        for ayahI in range(fromAyah - 1, toInThisSurah):
            surah, ayah, text = getVerse(f'{surahI + 1}:{ayahI + 1}')
            allVerses.append({
                'surah': surah,
                'ayah': ayah,
                'text': text
            })
    # calc accumulator
    calcAccumulator = CalculateAccumulator()
    allVerses = list(map(calcAccumulator.calc, allVerses))
    # get the middle verse
    middle = allVerses[-1]['accumulator'] / 2
    middle = snapAccumulator(allVerses, middle)
    return allVerses[middle]
getMiddle()

{'surah': 113,
 'ayah': 4,
 'text': 'وَمِنْ شَرِّ النَّفَّاثَاتِ فِي الْعُقَدِ',
 'accumulator': 105}

In [148]:
# read verses file
with open('versesPerPage.json', 'rb') as file:
    versesPerPage = json.load(file)
versesPerPage = dict(versesPerPage)

In [150]:

def addMiddle(page):
    pageMiddle = getMiddle(page[0]['verse_key'], page[-1]['verse_key'])
    pageMiddleVerse = f"{pageMiddle['surah']}:{pageMiddle['ayah']}"
    def pageM(verse):
        isMiddle = verse['verse_key'] == pageMiddleVerse
        if (isMiddle): verse['isMiddle'] = isMiddle
        return verse
    page = list(map(pageM, page))
    return page

newVersesPerPage = list(map(addMiddle, versesPerPage['pages']))

In [151]:
# edit verses file
with open('versesPerPage.json', 'w') as file:
    json.dump(newVersesPerPage, file)